In [40]:
#https://github.com/UKPLab/sentence-transformers/blob/master/examples/training_nli_bert.py

In [1]:
import urllib.request
import zipfile
import os
folder_path = '/Users/matthewvitha/Downloads'
print('Beginning download of datasets')

datasets = ['AllNLI.zip', 'stsbenchmark.zip', 'wikipedia-sections-triplets.zip']
server = "https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/datasets/"

for dataset in datasets:
    print("Download", dataset)
    url = server+dataset
    dataset_path = os.path.join(folder_path, dataset)
    urllib.request.urlretrieve(url, dataset_path)

    print("Extract", dataset)
    with zipfile.ZipFile(dataset_path, "r") as zip_ref:
        zip_ref.extractall(folder_path)
    os.remove(dataset_path)


print("All datasets downloaded and extracted")

Beginning download of datasets
Download AllNLI.zip
Extract AllNLI.zip
Download stsbenchmark.zip
Extract stsbenchmark.zip
Download wikipedia-sections-triplets.zip
Extract wikipedia-sections-triplets.zip
All datasets downloaded and extracted


In [3]:
!pip install sentence_transformers

     |████████████████████████████████| 51kB 1.6MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.3-cp37-none-any.whl size=57413 sha256=5ab649907d9f3c0235d778ff9eb9b772643a7e57d3f4b4c4ed37211fe4f34218
  Stored in directory: /Users/matthewvitha/Library/Caches/pip/wheels/20/89/88/95d8a3d4034bb77f52a6f66efdbfc623240221183dc0001c31
Successfully built sentence-transformers


In [4]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import *
import logging
from datetime import datetime

In [5]:
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

In [30]:
batch_size = 16
nli_reader = NLIDataReader('/Users/matthewvitha/Downloads/AllNLI')
sts_reader = STSDataReader('/Users/matthewvitha/Downloads/stsbenchmark')
train_num_labels = nli_reader.get_num_labels()
model_save_path = 'output1/training_nli_bert-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


In [7]:
nli_reader

In [8]:
# Use BERT for mapping tokens to embeddings
word_embedding_model = models.BERT('bert-base-uncased')

100%|██████████| 313/313 [00:00<00:00, 100869.55B/s]

2019-12-01 20:03:08 - https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache, downloading to /var/folders/73/w1kfkgh14vldbp72s104_r880000gn/T/tmpo1z95vv5
2019-12-01 20:03:09 - copying /var/folders/73/w1kfkgh14vldbp72s104_r880000gn/T/tmpo1z95vv5 to cache at /Users/matthewvitha/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
2019-12-01 20:03:09 - creating metadata file for /Users/matthewvitha/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
2019-12-01 20:03:09 - removing temp file /var/folders/73/w1kfkgh14vldbp72s104_r880000gn/T/tmpo1z95vv5
2019-12-01 20:03:09 - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /Users/matthewvitha/.cache/torch/


  0%|          | 0/440473133 [00:00<?, ?B/s]

2019-12-01 20:03:09 - https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-pytorch_model.bin not found in cache, downloading to /var/folders/73/w1kfkgh14vldbp72s104_r880000gn/T/tmpebw5jrj4


100%|██████████| 440473133/440473133 [04:47<00:00, 1592138.79B/s]


2019-12-01 20:07:56 - copying /var/folders/73/w1kfkgh14vldbp72s104_r880000gn/T/tmpebw5jrj4 to cache at /Users/matthewvitha/.cache/torch/pytorch_transformers/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
2019-12-01 20:07:57 - creating metadata file for /Users/matthewvitha/.cache/torch/pytorch_transformers/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
2019-12-01 20:07:57 - removing temp file /var/folders/73/w1kfkgh14vldbp72s104_r880000gn/T/tmpebw5jrj4
2019-12-01 20:07:57 - loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-pytorch_model.bin from cache at /Users/matthewvitha/.cache/torch/pytorch_transformers/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157


  0%|          | 0/231508 [00:00<?, ?B/s]

2019-12-01 20:07:59 - https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /var/folders/73/w1kfkgh14vldbp72s104_r880000gn/T/tmpm9a1yp2d


100%|██████████| 231508/231508 [00:00<00:00, 1278554.62B/s]


2019-12-01 20:08:00 - copying /var/folders/73/w1kfkgh14vldbp72s104_r880000gn/T/tmpm9a1yp2d to cache at /Users/matthewvitha/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2019-12-01 20:08:00 - creating metadata file for /Users/matthewvitha/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2019-12-01 20:08:00 - removing temp file /var/folders/73/w1kfkgh14vldbp72s104_r880000gn/T/tmpm9a1yp2d
2019-12-01 20:08:00 - loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/matthewvitha/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [9]:
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

In [10]:
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


2019-12-01 20:08:28 - Use pytorch device: cpu


In [16]:
# Convert the dataset to a DataLoader ready for training
logging.info("Read AllNLI train dataset")
train_data = SentencesDataset(nli_reader.get_examples('train.gz'), model=model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=train_num_labels)


2019-12-01 20:10:46 - Read AllNLI train dataset


Convert dataset: 100%|██████████| 942069/942069 [09:18<00:00, 1685.28it/s]

2019-12-01 20:20:12 - Num sentences: 942069
2019-12-01 20:20:12 - Sentences 0 longer than max_seqence_length: 0
2019-12-01 20:20:12 - Sentences 1 longer than max_seqence_length: 0
2019-12-01 20:20:12 - Softmax loss: #Vectors concatenated: 3


In [23]:
type(train_data)
print(len(train_data))

942069


In [18]:
train_data


In [35]:
len(dev_dataloader)

94

In [19]:
logging.info("Read STSbenchmark dev dataset")
dev_data = SentencesDataset(examples=sts_reader.get_examples('sts-dev.csv'), model=model)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=batch_size)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)

Convert dataset:  14%|█▍        | 210/1500 [00:00<00:00, 2097.11it/s]

2019-12-01 20:32:21 - Read STSbenchmark dev dataset


Convert dataset: 100%|██████████| 1500/1500 [00:01<00:00, 1344.59it/s]

2019-12-01 20:32:22 - Num sentences: 1500
2019-12-01 20:32:22 - Sentences 0 longer than max_seqence_length: 0
2019-12-01 20:32:22 - Sentences 1 longer than max_seqence_length: 0


In [31]:
# Configure the training
num_epochs = 1

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.00003) #10% of train data for warm-up, or my teenytiny train
logging.info("Warmup-steps: {}".format(warmup_steps))


                                                             

                                            



                                                       


                                                         
Iteration:   0%|          | 2/58880 [10:08<206:26:46, 12.62s/it]

Epoch:   0%|          | 0/1 [10:08<?, ?it/s]



Convert Evaluating:  45%|████▍     | 42/94 [04:47<02:50,  3.29s/it]


Iteration:   0%|          | 1/58880 [09:29<4214:16:14, 257.67s/it]

2019-12-01 20:43:02 - Warmup-steps: 2


In [32]:

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1,
          warmup_steps=warmup_steps,
          output_path=model_save_path
          )






Epoch:   0%|          | 0/1 [00:00<?, ?it/s]





Iteration:   0%|          | 0/58880 [00:00<?, ?it/s]
                                                             




                             





                                  

                                            



                                                       


                                                         
Iteration:   0%|          | 2/58880 [10:27<206:26:46, 12.62s/it]




Epoch:   0%|          | 0/1 [00:15<?, ?it/s]





Iteration:   0%|          | 0/58880 [00:15<?, ?it/s]

Epoch:   0%|          | 0/1 [10:27<?, ?it/s]



Convert Evaluating:  45%|████▍     | 42/94 [05:05<02:50,  3.29s/it]


Iteration:   0%|          | 1/58880 [09:47<4214:16:14, 257.67s/it]






Convert Evaluating:   0%|          | 0/94 [00:00<?, ?it/s]

2019-12-01 20:43:20 - Evaluation the model on  dataset in epoch 0 after 1 steps:









Convert Evaluating:   1%|          | 1/94 [00:00<01:29,  1.04it/s]






Convert Evaluating:   2%|▏         | 2/94 [00:01<01:27,  1.05it/s]






Convert Evaluating:   3%|▎         | 3/94 [00:03<01:49,  1.20s/it]






Convert Evaluating:   4%|▍         | 4/94 [00:04<01:46,  1.19s/it]






Convert Evaluating:   5%|▌         | 5/94 [00:06<02:08,  1.44s/it]






Convert Evaluating:   6%|▋         | 6/94 [00:08<02:21,  1.61s/it]






Convert Evaluating:   7%|▋         | 7/94 [00:10<02:14,  1.55s/it]






Convert Evaluating:   9%|▊         | 8/94 [00:12<02:18,  1.61s/it]






Convert Evaluating:  10%|▉         | 9/94 [00:13<02:11,  1.54s/it]






Convert Evaluating:  11%|█         | 10/94 [00:14<01:49,  1.31s/it]






Convert Evaluating:  12%|█▏        | 11/94 [00:14<01:36,  1.16s/it]






Convert Evaluating:  13%|█▎        | 12/94 [00:16<01:35,  1.16s/it]






Convert Evaluating:  14%|█▍        | 13/94 [00:17<01:29,  1.11s/it]






Convert Evaluating:  15%|█▍        | 14/

Convert Evaluating:  93%|█████████▎| 87/94 [03:04<00:10,  1.57s/it]






Convert Evaluating:  94%|█████████▎| 88/94 [03:06<00:09,  1.59s/it]






Convert Evaluating:  95%|█████████▍| 89/94 [03:07<00:07,  1.50s/it]






Convert Evaluating:  96%|█████████▌| 90/94 [03:08<00:05,  1.41s/it]






Convert Evaluating:  97%|█████████▋| 91/94 [03:09<00:03,  1.27s/it]






Convert Evaluating:  98%|█████████▊| 92/94 [03:11<00:02,  1.32s/it]






Convert Evaluating:  99%|█████████▉| 93/94 [03:12<00:01,  1.34s/it]






Convert Evaluating: 100%|██████████| 94/94 [03:13<00:00,  1.12s/it]
                                                             




                             





                                  

                                            



                                                       


                                                         
Iteration:   0%|          | 2/58880 [13:40<206:26:46, 12.62s/it]




Epoch:   0%|          | 0/1 [03:28<?, ?it/s]

2019-12-01 20:46:33 - Cosine-Similarity :	Pearson: 0.5917	Spearman: 0.5932
2019-12-01 20:46:33 - Manhattan-Distance:	Pearson: 0.6092	Spearman: 0.6146
2019-12-01 20:46:33 - Euclidean-Distance:	Pearson: 0.6060	Spearman: 0.6112
2019-12-01 20:46:33 - Dot-Product-Similarity:	Pearson: 0.3662	Spearman: 0.3580


...

2019-12-01 21:15:20 - Cosine-Similarity :	Pearson: 0.5642	Spearman: 0.5764
2019-12-01 21:15:20 - Manhattan-Distance:	Pearson: 0.5740	Spearman: 0.5776
2019-12-01 21:15:21 - Euclidean-Distance:	Pearson: 0.5650	Spearman: 0.5688
2019-12-01 21:15:21 - Dot-Product-Similarity:	Pearson: 0.4515	Spearman: 0.4549








Iteration:   0%|          | 11/58880 [32:16<2934:17:32, 179.44s/it]
                                                             




                             





                                                 

                                            



                                                       


                                                         
Iteration:   0%|          | 2/58880 [42:47<206:26:46, 12.62s/it]




Epoch:   0%|          | 0/1 [32:35<?, ?it/s]





Iteration:   0%|          | 11/58880 [32:35<2934:17:32, 179.44s/it]

Epoch:   0%|          | 0/1 [42:47<?, ?it/s]



Convert Evaluating:  45%|████▍     | 42/94 [37:25<02:50,  3.29s/it]


Iteration:   0%|          | 1/58880 [42:07<4214:16:14, 257.67s/it]






Convert Evaluating:   0%|          | 0/94 [00:00<?, ?it/s]

2019-12-01 21:15:40 - Evaluation the model on  dataset in epoch 0 after 12 steps:









Convert Evaluating:   1%|          | 1/94 [00:00<01:20,  1.15it/s]






Convert Evaluating:   2%|▏         | 2/94 [00:01<01:18,  1.18it/s]






Convert Evaluating:   3%|▎         | 3/94 [00:02<01:20,  1.12it/s]






Convert Evaluating:   4%|▍         | 4/94 [00:03<01:18,  1.15it/s]






Convert Evaluating:   5%|▌         | 5/94 [00:04<01:19,  1.11it/s]






Convert Evaluating:   6%|▋         | 6/94 [00:05<01:24,  1.04it/s]






Convert Evaluating:   7%|▋         | 7/94 [00:06<01:29,  1.03s/it]






Convert Evaluating:   9%|▊         | 8/94 [00:07<01:29,  1.05s/it]






Convert Evaluating:  10%|▉         | 9/94 [00:09<01:33,  1.09s/it]






Convert Evaluating:  11%|█         | 10/94 [00:09<01:21,  1.03it/s]






Convert Evaluating:  12%|█▏        | 11/94 [00:10<01:16,  1.08it/s]






Convert Evaluating:  13%|█▎        | 12/94 [00:11<01:24,  1.03s/it]






Convert Evaluating:  14%|█▍        | 13/94 [00:12<01:17,  1.04it/s]






Convert Evaluating:  15%|█▍        | 14/

KeyboardInterrupt: 

In [39]:
evaluator.csv_headers

['epoch',
 'steps',
 'cosine_pearson',
 'cosine_spearman',
 'euclidean_pearson',
 'euclidean_spearman',
 'manhattan_pearson',
 'manhattan_spearman',
 'dot_pearson',
 'dot_spearman']

In [34]:
len(evaluator)

TypeError: object of type 'EmbeddingSimilarityEvaluator' has no len()